In [45]:
import numpy as np

In [46]:
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import PanTool, BoxZoomTool, HoverTool, CrosshairTool, ResetTool

In [47]:
from sklearn.decomposition import PCA

In [48]:
from PIL import Image
import base64
from io import BytesIO

In [49]:
def make_scatter2d_images(x, y, names=None, image_files=None, clustering=None):
    source_data = dict(x=x, y=y)
    if names is not None:
        source_data["desc"] = names
        tooltips_desc = """<span style="font-size: 17px; font-weight: bold;">@desc</span>"""
    else:
        tooltips_desc = ""
    if image_files is not None:
        source_data["imgs"] = image_files
        tooltips_images = """
            <div>
                <img
                    src="@imgs" height="42" alt="@imgs" width="42"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
        """
    else:
        tooltips_images = ""
    if clustering is not None:
        color_map = make_color_map(clustering)
        cluster_colors = [color_map[c] for c in clustering]
        source_data['cluster_color'] = cluster_colors
    source = ColumnDataSource(data=source_data)
    hover = HoverTool(tooltips="""
        <div>
            {}
            <div>
                {}
                <span style="font-size: 15px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """.format(tooltips_images, tooltips_desc))
    p = figure(width=600, height=600)
    for t in [PanTool(), BoxZoomTool(), hover, CrosshairTool(), ResetTool()]:
        p.add_tools(t)
    if clustering is not None:
        p.circle(x='x', y='y',
                 fill_color='cluster_color',
                 line_color='cluster_color',
                 size=5, source=source)
    else:
        p.circle(x='x', y='y', size=5, source=source)
    return p


In [38]:
def gnp2im(image_np, bit_depth_scale_factor):
    """Converts an image stored as a 2-D grayscale Numpy array into a PIL image."""
    return Image.fromarray((image_np * bit_depth_scale_factor).astype(np.uint8), mode='L')

In [39]:
def to_base64(png):
    return "data:image/png;base64," + base64.b64encode(png).decode("utf-8")

In [31]:
output_notebook()

Loading BokehJS ...

In [32]:
n = 32
h = 32
w = 32

In [33]:
data = np.random.rand(n, h, w)

In [34]:
pca = PCA(n_components=2)

In [35]:
z = pca.fit_transform(data.reshape((n, h * w)))

In [36]:
z.shape

(32, 2)

In [37]:
x = z[:, 0]
y = z[:, 1]

In [40]:
bit_depth_scale_factor = 255

In [42]:
thumbnails = []
for gnp in data:
    im = gnp2im(gnp, bit_depth_scale_factor)
    memout = BytesIO()
    im.save(memout, format='png')
    thumbnails.append(to_base64(memout.getvalue()))

In [43]:
p_scatter = make_scatter2d_images(x, y, names=None, image_files=thumbnails, clustering=None)

In [44]:
show(row(p_scatter,), notebook_handle=True)